In [6]:
!pip install langchain
!pip install langchain-community
!pip install pypdf
!pip install sentence-transformers
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 14.5 MB/s eta 0:00:0000:0100:01


In [168]:
import os
import json
import boto3
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, GenerationConfig, AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline
from langchain_community.llms import SagemakerEndpoint
from langchain_community.llms.sagemaker_endpoint import LLMContentHandler
from typing import Dict
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


import warnings
warnings.filterwarnings('ignore')

In [169]:
# loaders = [
#     PyPDFLoader("docs/uber_2021.pdf"),
#     PyPDFLoader("docs/lyft_2021.pdf")
# ]

file_name1 = "MIZUHO_FINANCIAL_GROUP_1.pdf"
file_name2 = "MIZUHO_FINANCIAL_GROUP_2.pdf"

loaders = [
    PyPDFLoader("docs/MIZUHO_FINANCIAL_GROUP_1.pdf"),
    PyPDFLoader("docs/MIZUHO_FINANCIAL_GROUP_2.pdf")
]

docs = []
for loader in loaders:
    docs.extend(loader.load())
    
print(len(docs))

51


In [190]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1300,
    chunk_overlap = 150
)

splits = text_splitter.split_documents(docs)
print("Done")

Done


In [191]:
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_db = FAISS.from_documents(documents=splits, embedding=embedding)
print("Done")

Done


In [192]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": prompt, "parameters": model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generated_text"]


content_handler = ContentHandler()

runtime = boto3.client("runtime.sagemaker")

llm=SagemakerEndpoint(
        endpoint_name="mistral-llm",
        client=runtime,
        model_kwargs={"temperature": 0.5},
        content_handler=content_handler
    )
print("Done")

Done


In [193]:

template = """
Answer the question as detailed as possible from the provided context, make sure to provide all the details.
If the answer is not in provided context just say, "Answer is not available in the context.", please don't provide the wrong answer.

Provide Confidence Score as well.

Context:\n {context}?\n

Question: \n Get {question}\n

Confidence Score:"""

prompt = PromptTemplate(template = template, input_variables = ["context", "question"])

chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = vector_db.as_retriever(search_kwargs={"k":12}),
    return_source_documents=False,
    chain_type = "stuff",
    chain_type_kwargs={"prompt":prompt}
)
print("Done")

Done


In [194]:

question = "Fixing of Record Date" + " from "+file_name1
response_file1 = chain({"query":question})

question = "Fixing of Record Date" + " from "+file_name2
response_file2 = chain({"query":question})

response_file1_string = response_file1.get("result")
response_file2_string = response_file2.get("result")


sub1 = "Question:"

response_file1_output = "Question:\n"
response_file2_output = "Question:\n"

response_file1_output = response_file1_output + response_file1_string.rsplit(sub1, 1)[1]
print(response_file1_output)

print("\n\n-------------------------")

response_file2_output = response_file2_output + response_file2_string.rsplit(sub1, 1)[1]
print(response_file2_output)

Question:
 
 Get Fixing of Record Date from MIZUHO_FINANCIAL_GROUP_1.pdf


Confidence Score: 95%

Answer: The record date is the date fixed by the Depositary for the determination of the Owners who are entitled to receive a dividend, distribution, or rights, or the net proceeds of sale thereof, give voting instructions, or are responsible for any fee assessed by the Depositary. The record date can be fixed whenever any cash dividend or other cash distribution becomes payable, any distribution other than cash is made, rights are issued


-------------------------
Question:
 
 Get Fixing of Record Date from MIZUHO_FINANCIAL_GROUP_2.pdf


Confidence Score: 100%

Answer: Based on the provided context, the Depositary, Mizuho Financial Group, Inc., or The Bank of New York has the authority to fix a record date for various events related to the American Depositary Shares (ADS) of Mizuho Financial Group, Inc. These events include the determination of Owners entitled to receive cash dividends, 

In [195]:
def convert_to_json_reponse(str_input):
    internal_json = '''
        "Question":{},
        "Confidence Score":{},
        "Answer":{}
    '''
    # slicing question
    sub1 = "Question:"
    sub2 = "Confidence Score:"
 
    # getting elements in between using split() and join()
    question = ''.join(str_input.split(sub1)[1].split(sub2)[0]).strip()
    question = "\""+question+"\""
    
    # slicing Confidence Score
    sub1 = "Confidence Score:"
    sub2 = "Answer:"
 
    # getting elements in between using split() and join()
    conf_score = ''.join(str_input.split(sub1)[1].split(sub2)[0]).strip()
    conf_score = "\""+conf_score+"\""
    
    # slicing answer
    sub1 = "Answer:"
 
    # getting elements in between using split() and join()
    answer = ''.join(str_input.split(sub1)[1]).strip()
    answer = "\""+answer+"\""
    
    result = internal_json.format(question, conf_score, answer)
    
    result = "{"+result+"}"
    
    return result

# print(convert_to_json_reponse(response_file1_output))

json_response = '''
"{}":{},
"{}":{}
'''

result = json_response.format(file_name1, convert_to_json_reponse(response_file1_output), file_name2, convert_to_json_reponse(response_file2_output))
result = "{"+result+"}"

print(result)


{
"MIZUHO_FINANCIAL_GROUP_1.pdf":{
        "Question":"Get Fixing of Record Date from MIZUHO_FINANCIAL_GROUP_1.pdf",
        "Confidence Score":"95%",
        "Answer":"The record date is the date fixed by the Depositary for the determination of the Owners who are entitled to receive a dividend, distribution, or rights, or the net proceeds of sale thereof, give voting instructions, or are responsible for any fee assessed by the Depositary. The record date can be fixed whenever any cash dividend or other cash distribution becomes payable, any distribution other than cash is made, rights are issued"
    },
"MIZUHO_FINANCIAL_GROUP_2.pdf":{
        "Question":"Get Fixing of Record Date from MIZUHO_FINANCIAL_GROUP_2.pdf",
        "Confidence Score":"100%",
        "Answer":"Based on the provided context, the Depositary, Mizuho Financial Group, Inc., or The Bank of New York has the authority to fix a record date for various events related to the American Depositary Shares (ADS) of Mizuho Fin

In [198]:
print(type(result))

json_result = json.loads(result)

print(type(json_result))
json_result

<class 'str'>
<class 'dict'>


{'MIZUHO_FINANCIAL_GROUP_1.pdf': {'Question': 'Get Fixing of Record Date from MIZUHO_FINANCIAL_GROUP_1.pdf',
  'Confidence Score': '95%',
  'Answer': 'The record date is the date fixed by the Depositary for the determination of the Owners who are entitled to receive a dividend, distribution, or rights, or the net proceeds of sale thereof, give voting instructions, or are responsible for any fee assessed by the Depositary. The record date can be fixed whenever any cash dividend or other cash distribution becomes payable, any distribution other than cash is made, rights are issued'},
 'MIZUHO_FINANCIAL_GROUP_2.pdf': {'Question': 'Get Fixing of Record Date from MIZUHO_FINANCIAL_GROUP_2.pdf',
  'Confidence Score': '100%',
  'Answer': 'Based on the provided context, the Depositary, Mizuho Financial Group, Inc., or The Bank of New York has the authority to fix a record date for various events related to the American Depositary Shares (ADS) of Mizuho Financial Group, Inc. These events include